# El rol de Dwight Schrute en 'The Office'

En apariencia, la serie *The Office* se presenta como una comedia de conjunto sobre los empleados de Dunder Mifflin. Sin embargo, mi teoría es que, con el tiempo, la narrativa evoluciona dándole más peso narrativo a un personaje en particular: Dwight Schrute.

Mediante el análisis de datos sobre su presencia en episodios, sus diálogos, su impacto en las valoraciones y la evolución de sus relaciones, intentaremos demostrar que Dwight es el verdadero protagonista y el epicentro de la serie.

Hipótesis principal: 
> Dwight es el verdadero protagonista de la serie.


Hipótesis secundarias:
1.  Dwight aparece en más episodios que otros personajes.
2.  Dwight tiene más diálogo que el resto de sus compañeros, especialmente a medida que la serie avanza.
3.  Episodios con menos presencia de Dwight tienen peores ratings.
4.  El círculo social de Dwight crece el que más. *comparte más escenas
5.  El contenido de sus diálogos y monólogos muestra el mayor arco de personaje.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Fuentes de datos

Estos son los datasets de los que parte nuestro análisis:

| Nombre Dataset | Link |
| :--- | :--- |
| The Office Dataset | [https://www.kaggle.com/datasets/nehaprabhavalkar/the-office-dataset](https://www.kaggle.com/datasets/nehaprabhavalkar/the-office-dataset) |
| The Office Lines | [https://www.kaggle.com/datasets/fabriziocominetti/the-office-lines](https://www.kaggle.com/datasets/fabriziocominetti/the-office-lines) |
| The Office Episodes Data | [https://www.kaggle.com/datasets/bcruise/the-office-episodes-data](https://www.kaggle.com/datasets/bcruise/the-office-episodes-data) |
| The Office (US) Complete Dialogue/Transcript | [https://www.kaggle.com/datasets/nasirkhalid24/the-office-us-complete-dialoguetranscript](https://www.kaggle.com/datasets/nasirkhalid24/the-office-us-complete-dialoguetranscript) |



Cargamos y exploramos los dos datasets principales de episodios para entender su estructura y decidir cuál usar.

In [ ]:
episodios1 = pd.read_csv("data/the_office_series.csv")
episodios1.info()
episodios1

In [ ]:
episodios2 = pd.read_csv("data/the_office_imdb.csv")
episodios2.info()
episodios2


In [ ]:
lines1 = pd.read_csv("data/the-office_lines.csv")
lines1.info()
lines1 #contiene [direcciones]

In [ ]:
lines2 = pd.read_csv("data/The-Office-Lines-V4.csv")
lines2.info()
lines2 # 4k líneas menos(?) pero con escenas

## Limpieza

In [ ]:
# al final solo usamos una
episodes = episodios2[['season', 'title', 'imdb_rating']]
episodes

In [ ]:
# eliminar innecesaria
lines = lines1.drop(columns=['Unnamed: 0'])
lines

In [ ]:
# estandarizar títulos
lines2['title'] = lines2['title'].str.rstrip(' (Parts 1&2)')

In [ ]:
# seleccionar columnas útiles
scenes = lines2[['season', 'episode', 'title', 'scene', 'speaker']]
scenes

## Desarrollo

### H1. Dwight aparece en más episodios que otros personajes

In [ ]:
# seleccionar valores únicos de lines1
df = lines1[['Season', 'Episode_Number', 'Character']].drop_duplicates()

# agrupar por personaje
epXpers = df.groupby("Character")["Episode_Number"].count().sort_values(ascending=False).head(5)
epXpers

In [ ]:
# crear gráfico

plt.figure(figsize=(8, 4))
plt.hlines(y=epXpers.index,
           xmin=170,
           xmax=epXpers,
           color='skyblue')
plt.plot(epXpers, epXpers.index, "o");

# añadir count a cada punta
for i, count in enumerate(epXpers):
    plt.text(count + 0.1, i, f'{count}', va='center', ha='left', fontsize=11)

plt.xticks([])

plt.xlabel('Número de episodios', fontsize=12)
plt.ylabel('Personajes', fontsize=12)
plt.title('Número de Episodios con cada Personaje', fontsize=14, fontweight='bold')
plt.tight_layout()

"""
plt.savefig('episodios_por_personaje.png', dpi=300, bbox_inches='tight')
print("Gráfico guardado como 'episodios_por_personaje.png'")
"""

Parece que sí se cumple, de hecho es el que en más episodios está.

### H2. Dwight tiene más diálogo que el resto de sus compañeros, especialmente a medida que la serie avanza.

In [ ]:
# contar num líneas por personaje
nLin = lines.Character.value_counts().head(10)
nLin

In [ ]:
# crear gráfico
plt.figure(figsize=(8, 5))

bars = plt.bar(nLin.index, nLin.values)

"""
# Guardar el gráfico
plt.savefig('lineas_por_personaje.png', dpi=300, bbox_inches='tight')
print("Gráfico guardado como 'lineas_por_personaje.png'")
"""

Es uno de los personajes con más diálogo. Habría que hacer más análisis para ver cómo evoluciona a lo largo de diferentes temporadas.

### H3. 3.  Episodios con menos presencia de Dwight tienen peores ratings

In [ ]:
# número de lineas de Dwight en el episodio
dwight_lines = lines2[lines2['speaker'] == 'Dwight'].groupby(['season', 'title']).size().reset_index(name='dwight_line_count')

# juntar con los ratings
merged_df = pd.merge(dwight_lines, episodes, on=['season', 'title'])

# calcular correlacion
correlation = merged_df['dwight_line_count'].corr(merged_df['imdb_rating'])

print("CORRELACIÓN LÍNEAS DE DWIGHT VS RATING")
print(f"Correlation coefficient: {correlation:.3f}")
print("\n")

In [ ]:
# crear scatterplot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=merged_df, x='dwight_line_count', y='imdb_rating')
sns.regplot(data=merged_df, x='dwight_line_count', y='imdb_rating',
            scatter=False, color='red')

plt.text(75, 8.25, f'r = {correlation:.3f}',
         fontsize=12, color='red', weight='bold')

plt.title('Líneas de Dwight vs Rating IMDb')
plt.xlabel('Número de líneas de Dwight')
plt.ylabel('Rating IMDb')

# Guardar el gráfico
plt.savefig('dwight_lines_vs_rating.png', dpi=300, bbox_inches='tight')
print("Gráfico guardado como 'dwight_lines_vs_rating.png'")


No hay correlación aparente. Sería interesante ver cómo la cantidad del diálogo o su relevancia afectan al resultado, así como controlar  otros factores como los escritores y dirección del episodio o si es un episodio especial.

Nota: Parece haber un bug que no he conseguido solucionar - en el notebook el gráfico queda diferente y con más datos.

### H4.  El círculo social de Dwight crece el que más.

In [ ]:
# seleccionar los personajes con mayores líneas
top_speakers = scenes.speaker.value_counts().head(10).index
top_speakers

In [ ]:
# crear gráfico

# lista de personajes para mostrar en el gráfico
characters = top_speakers

# definir temporadas
seasons = sorted(scenes['season'].unique())

plt.figure(figsize=(12, 6))

# iterar la lista de personajes creando su línea de progreso
for character in characters:
    character_diversity = []
    
    for season in seasons:
        # escenas en las que el personaje aparece
        char_scenes = scenes[
            (scenes['season'] == season) & 
            (scenes['speaker'] == character)
        ]['scene'].unique()
        
        # personajes con los que comparte escena
        co_speakers = scenes[
            (scenes['season'] == season) & 
            (scenes['scene'].isin(char_scenes)) & 
            (scenes['speaker'] != character)
        ]['speaker'].unique()
        
        character_diversity.append(len(co_speakers))
    
    # linea del personaje
    plt.plot(seasons, character_diversity, marker='o', linewidth=2, markersize=6, label=character)

plt.xlabel('Temporadas')
plt.ylabel('Número de personajes con los que interactúa')
plt.title('Número de interacciones con otros personajes')
plt.grid(True, alpha=0.3)
plt.xticks(seasons)
plt.legend()

"""# Guardar el gráfico
plt.savefig('interacciones_por_personaje.png', dpi=300, bbox_inches='tight')
print("Gráfico guardado como 'interacciones_por_personaje.png'")
"""

Parece que sí hay una tendencia positiva que no todos los personajes comparten por igual.

### H5.  El contenido de sus diálogos y monólogos muestra el mayor arco de personaje.

Demasiado ambicioso para esta primera exploración, pero tenemos los datos a la espera de más análisis :)

## Conclusión

Aunque los resultados no fueron perfectos en todo, la evidencia apunta a una cosa: Dwight terminó siendo mucho más importante de lo que nadie esperaba. De ser el raro de la oficina, se convirtió en un personaje clave sin el cual la serie no hubiera sido lo mismo.